Open Play Shots - Expected Miss Model - Model Tuning and Building - GBM

In [61]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [63]:
model_version = 4
model_name = 'expected_miss_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [64]:
RESPONSE = ModellingDataContract.RESPONSE_MISS

In [65]:
FEATURES = ModellingDataContract.open_goal_modelling_feature_list

In [66]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_open_goal

Load Data

In [67]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,0,1,107.0,32.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,True,False,False,True,False,False,True,False,False
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,113.0,33.0,80.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,False,False,True,False,False,True,True,False,False
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,119.0,39.0,80.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,True,False,False,True,False,False,True,False,False
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,125.0,45.0,80.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,False,False,True,True,False,False,True,False,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,137.0,45.0,92.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,True,False,False,False,False,True,True,False,False


In [68]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [69]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [70]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [92]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.2072240122095981, 0.20715835140997832, 0.20745762711864407)

In [71]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
6,46,possGain,goal,476.0,1,1383,1389.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,56.0,6.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,19.0,9.0,10.0,Kick,Handball Received,Handball,Handball Received,56.0,58.0,58.0,58.0,6.0,21.0,25.0,26.0,1389.0,1389.0,1388.0,1387.0,6.0,0.0,15.132746,22.0,6.0,22.803509,0.266252,15.255119,0.276208,15.825576,0.987285,56.567262,484.0,3.091042,False,True,False,False,False,True,False,False,True,False,0,0,0,1,0
7,50,possGain,behind,512.0,1,1462,1500.0,Brisbane Lions,Brisbane Lions,Oscar McInerney,Oscar_McInerney,Kick,65.0,-15.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,1,1,1,0,19.0,15.0,4.0,Kick,Hard Ball Get Crumb,Spoil,Kick Inside 50 Result,65.0,62.0,60.0,60.0,-15.0,-16.0,-17.0,-17.0,1500.0,1499.0,1497.0,1496.0,38.0,1.0,3.162278,13.0,15.0,19.849433,0.856706,49.085617,0.216802,12.421849,0.826928,47.379494,169.0,2.564949,False,True,False,False,True,False,False,False,True,False,0,0,0,1,0
8,79,possGain,goal,747.0,2,709,720.0,Sydney,Sydney,Sam Wicks,Sam_Wicks,Kick,41.0,18.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,20.0,40.0,-20.0,Kick,Handball Received,Handball,Handball Received,41.0,38.0,37.0,36.0,18.0,16.0,15.0,15.0,720.0,720.0,719.0,718.0,11.0,0.0,3.605551,37.0,18.0,41.146081,0.452778,25.942295,0.140721,8.062731,0.443767,25.425977,1369.0,3.610918,False,True,False,False,False,True,False,False,True,False,0,0,0,1,0
13,96,centreBounce,goal,899.0,2,1368,1381.0,Brisbane Lions,Brisbane Lions,Lincoln McCarthy,Lincoln_McCarthy,Kick,38.0,7.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,6,0,38.0,46.0,-8.0,Kick,Loose Ball Get,Spoil,Kick Inside 50 Result,38.0,39.0,46.0,46.0,7.0,4.0,-1.0,-1.0,1381.0,1379.0,1378.0,1376.0,13.0,2.0,3.162278,40.0,7.0,40.607881,0.173246,9.926246,0.156216,8.950499,0.493307,28.264407,1600.0,3.688879,False,True,False,False,False,False,True,False,True,False,0,1,0,0,0
16,141,centreBounce,behind,1250.0,3,901,912.0,Brisbane Lions,Brisbane Lions,Keidean Coleman,Keidean_Coleman,Kick,36.0,-31.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,1,1,1,0,57.0,71.0,-14.0,Kick,Hard Ball Get,Contest Target,Ground Kick,36.0,32.0,29.0,6.0,-31.0,-32.0,-31.0,-2.0,912.0,910.0,909.0,906.0,11.0,2.0,4.123106,42.0,31.0,52.201533,0.635838,36.430858,0.099014,5.673100,0.306285,17.548851,1764.0,3.737670,False,False,True,False,True,False,False,False,True,False,0,1,0,0,0


Preprocess Data

In [72]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [73]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [74]:
X_train_preproc.shape, X_test_preproc.shape

((5897, 17), (1844, 17))

In [75]:
X_train_preproc.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0
3,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0
4,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0
5,27.0,28.0,34.0,35.0,3.162278,59.682493,0.546167,0.091898,0,0,0,1,0,51.0,31.0,45.0,1.0
9,46.0,41.0,35.0,30.0,5.000000,32.062439,0.062419,0.201226,0,0,0,0,1,32.0,2.0,13.0,0.0


In [76]:
X_test_preproc.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
6,56.0,58.0,58.0,58.0,15.132746,22.803509,0.266252,0.276208,0,0,0,1,0,22.0,6.0,6.0,0.0
7,65.0,62.0,60.0,60.0,3.162278,19.849433,0.856706,0.216802,0,0,0,1,0,13.0,15.0,38.0,1.0
8,41.0,38.0,37.0,36.0,3.605551,41.146081,0.452778,0.140721,0,0,0,1,0,37.0,18.0,11.0,0.0
13,38.0,39.0,46.0,46.0,3.162278,40.607881,0.173246,0.156216,0,1,0,0,0,40.0,7.0,13.0,2.0
16,36.0,32.0,29.0,6.0,4.123106,52.201533,0.635838,0.099014,0,1,0,0,0,42.0,31.0,11.0,2.0


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [77]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [78]:
xgb_tuner.training_data.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0
3,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0
4,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0
5,27.0,28.0,34.0,35.0,3.162278,59.682493,0.546167,0.091898,0,0,0,1,0,51.0,31.0,45.0,1.0
9,46.0,41.0,35.0,30.0,5.000000,32.062439,0.062419,0.201226,0,0,0,0,1,32.0,2.0,13.0,0.0


In [79]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-11 13:57:32,475] A new study created in memory with name: no-name-6e17ffa7-dd3b-434e-a44c-a691692d6815
[I 2023-07-11 13:57:32,553] Trial 0 finished with value: 0.6359191832917379 and parameters: {'max_depth': 15, 'min_child_weight': 36, 'eta': 0.019811048972429544, 'gamma': 0.00012875908552887553, 'lambda': 2.182648205037418, 'alpha': 0.0014272381807032546, 'subsample': 0.26069488938502205, 'colsample_bytree': 0.6401666308012914}. Best is trial 0 with value: 0.6359191832917379.
[I 2023-07-11 13:57:32,798] Trial 1 finished with value: 0.6418667334216996 and parameters: {'max_depth': 17, 'min_child_weight': 3, 'eta': 0.8375659519329836, 'gamma': 0.00043023163436495807, 'lambda': 0.003894185920446439, 'alpha': 0.4551432590428881, 'subsample': 0.606891435672837, 'colsample_bytree': 0.5326265651031585}. Best is trial 0 with value: 0.6359191832917379.
[I 2023-07-11 13:57:32,821] Trial 2 finished with value: 0.5013512192813119 and parameters: {'max_depth': 3, 'min_child_weight': 16

Number of finished trials:  1000
Best trial:
  Value: 0.45010344446664713
  Params: 
    max_depth: 17
    min_child_weight: 82
    eta: 0.7295722418947294
    gamma: 0.0001704251384576935
    lambda: 0.00484268231985966
    alpha: 0.0034528035638621674
    subsample: 0.48655292014343454
    colsample_bytree: 0.3797636923566364


In [80]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 17,
 'min_child_weight': 82,
 'eta': 0.7295722418947294,
 'gamma': 0.0001704251384576935,
 'lambda': 0.00484268231985966,
 'alpha': 0.0034528035638621674,
 'subsample': 0.48655292014343454,
 'colsample_bytree': 0.3797636923566364}

Training Model - SuperXGBClassifier class for training and predictions

In [81]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [82]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [83]:
super_xgb.fit()

[0]	validation_0-logloss:0.52631	validation_1-logloss:0.52670
[1]	validation_0-logloss:0.50254	validation_1-logloss:0.50824
[2]	validation_0-logloss:0.49806	validation_1-logloss:0.50430
[3]	validation_0-logloss:0.48876	validation_1-logloss:0.49326
[4]	validation_0-logloss:0.48774	validation_1-logloss:0.49289
[5]	validation_0-logloss:0.48725	validation_1-logloss:0.49215
[6]	validation_0-logloss:0.48637	validation_1-logloss:0.49259
[7]	validation_0-logloss:0.48551	validation_1-logloss:0.49134
[8]	validation_0-logloss:0.48490	validation_1-logloss:0.49184
[9]	validation_0-logloss:0.48482	validation_1-logloss:0.49282
[10]	validation_0-logloss:0.48460	validation_1-logloss:0.49417
[11]	validation_0-logloss:0.48303	validation_1-logloss:0.49214
[12]	validation_0-logloss:0.48265	validation_1-logloss:0.49335
[13]	validation_0-logloss:0.48244	validation_1-logloss:0.49435
[14]	validation_0-logloss:0.48190	validation_1-logloss:0.49292
[15]	validation_0-logloss:0.48129	validation_1-logloss:0.48937
[1

In [84]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3797636923566364, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0001704251384576935, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.72...
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_leaves=None,
              min_child_weight=82, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': -1,
                                    'Distance_Since_Last_Action': 1,
                                    'Distance_to_Middle_Goal': -1,
                                    'Visible_Goal_Angle': 1, 'x0': -1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [85]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'x1',
 'x2',
 'x3',
 'Distance_Since_Last_Action',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle',
 'ballUp',
 'centreBounce',
 'kickIn',
 'possGain',
 'throwIn',
 'Distance_to_Right_Goal_x',
 'Distance_to_Middle_y',
 'Chain_Duration',
 'Time_Since_Last_Action']

In [86]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [87]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [88]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [89]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [90]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.20891851, 0.2072240122095981, 0.20589940753860225)

In [91]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.20602845, 0.20715835140997832, 0.20359363542675854)

Export model

In [93]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [94]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,1,11.0,0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0,0,0.237190,0.233659
1,202101_BrisbaneLions_Sydney,21,222.0,0,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0,0,0.190383,0.197524
2,202101_BrisbaneLions_Sydney,38,392.0,1,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0,0,0.130665,0.145741
3,202101_BrisbaneLions_Sydney,43,444.0,0,27.0,28.0,34.0,35.0,3.162278,59.682493,0.546167,0.091898,0,0,0,1,0,51.0,31.0,45.0,1.0,0,0.226515,0.225725
4,202101_BrisbaneLions_Sydney,83,781.0,0,46.0,41.0,35.0,30.0,5.000000,32.062439,0.062419,0.201226,0,0,0,0,1,32.0,2.0,13.0,0.0,0,0.095251,0.111425


In [95]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,46,476.0,0,56.0,58.0,58.0,58.0,15.132746,22.803509,0.266252,0.276208,0,0,0,1,0,22.0,6.0,6.0,0.0,0,0.098804,0.115006
1,202101_BrisbaneLions_Sydney,50,512.0,0,65.0,62.0,60.0,60.0,3.162278,19.849433,0.856706,0.216802,0,0,0,1,0,13.0,15.0,38.0,1.0,0,0.222086,0.222382
2,202101_BrisbaneLions_Sydney,79,747.0,0,41.0,38.0,37.0,36.0,3.605551,41.146081,0.452778,0.140721,0,0,0,1,0,37.0,18.0,11.0,0.0,0,0.168960,0.179742
3,202101_BrisbaneLions_Sydney,96,899.0,0,38.0,39.0,46.0,46.0,3.162278,40.607881,0.173246,0.156216,0,1,0,0,0,40.0,7.0,13.0,2.0,0,0.272536,0.258748
4,202101_BrisbaneLions_Sydney,141,1250.0,0,36.0,32.0,29.0,6.0,4.123106,52.201533,0.635838,0.099014,0,1,0,0,0,42.0,31.0,11.0,2.0,0,0.271283,0.257888


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")